In [1]:
import os
import pickle

save_path="D:\\study\\sugyeong_github\\FingerVein-Spoofing\\"

print('>> train')
with open(save_path+"train.pickle", mode='rb') as f:
    trainDict = pickle.load(f)

train_X = trainDict['freq_signal']
train_y = trainDict['class'].ravel()
print(train_X.shape)
print(train_y.shape)

print()
print('>> test')
with open(save_path+"test.pickle", mode='rb') as f:
    testDict = pickle.load(f)

test_X = testDict['freq_signal']
test_y = testDict['class'].ravel()
print(test_X.shape)
print(test_y.shape)

>> train
(580, 750)
(580,)

>> test
(562, 750)
(562,)


In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Dense, Activation, Flatten, Conv1D, MaxPooling1D, Dropout, BatchNormalization, LeakyReLU
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from sklearn.model_selection import train_test_split
import numpy as np

## data shuffle

In [3]:
import random

tmp = [[x,y] for x, y in zip(train_X,train_y)]
random.shuffle(tmp)
train_X = [n[0] for n in tmp]
train_y = [n[1] for n in tmp]

train_y=np.array(train_y)
print(train_y)

[1 0 0 1 1 1 1 1 0 1 1 1 1 0 1 0 1 1 0 1 1 1 1 1 0 0 1 1 0 0 1 0 1 0 1 0 1
 0 0 1 1 1 1 1 0 1 0 0 1 1 1 0 0 0 1 1 0 1 1 0 0 1 1 0 1 0 0 1 0 1 0 0 1 1
 1 1 0 0 0 1 1 0 0 0 0 1 1 0 1 0 0 1 0 0 0 1 1 0 0 1 1 1 0 1 1 0 1 1 1 1 1
 0 0 0 0 0 1 1 1 0 0 0 0 1 0 0 1 0 0 1 0 0 0 0 0 0 1 0 1 1 1 0 0 0 0 1 0 0
 1 0 0 1 0 1 0 1 1 1 0 1 0 0 1 0 1 1 0 1 1 1 1 0 1 1 0 0 1 0 1 1 0 1 1 0 1
 0 1 0 0 1 0 1 1 1 0 0 0 1 0 0 1 1 0 1 0 0 1 0 1 1 0 0 0 1 1 0 0 1 1 0 0 1
 0 0 0 1 0 0 0 1 0 1 1 1 0 0 0 0 0 0 0 1 1 0 0 0 1 1 0 1 0 1 0 0 1 1 0 0 1
 1 1 0 1 1 0 0 1 0 0 1 0 0 1 0 0 0 0 1 0 1 1 1 1 1 1 0 1 0 0 0 0 1 1 0 1 1
 1 0 1 0 0 0 1 1 1 0 1 1 1 1 0 1 1 0 1 0 1 1 0 0 0 0 1 0 1 0 1 1 0 0 0 1 0
 1 1 0 0 0 0 0 0 1 0 1 0 0 0 1 0 1 1 0 1 1 0 1 1 1 1 1 1 0 0 0 1 0 0 0 0 1
 0 0 0 1 0 1 0 1 0 0 0 0 1 0 1 1 0 1 1 0 1 1 0 0 0 0 1 0 0 0 1 1 1 1 1 1 1
 1 1 0 1 0 1 0 0 1 0 1 1 1 1 0 1 1 0 1 1 1 1 0 1 1 1 1 1 0 0 0 1 1 0 1 1 1
 1 0 1 1 0 0 0 1 1 1 0 0 1 0 0 0 1 0 1 0 1 1 1 1 0 0 0 1 0 1 0 0 0 0 0 1 1
 1 0 1 0 1 1 0 1 0 0 0 0 

# Data 확인

In [4]:
print(train_y)

[1 0 0 1 1 1 1 1 0 1 1 1 1 0 1 0 1 1 0 1 1 1 1 1 0 0 1 1 0 0 1 0 1 0 1 0 1
 0 0 1 1 1 1 1 0 1 0 0 1 1 1 0 0 0 1 1 0 1 1 0 0 1 1 0 1 0 0 1 0 1 0 0 1 1
 1 1 0 0 0 1 1 0 0 0 0 1 1 0 1 0 0 1 0 0 0 1 1 0 0 1 1 1 0 1 1 0 1 1 1 1 1
 0 0 0 0 0 1 1 1 0 0 0 0 1 0 0 1 0 0 1 0 0 0 0 0 0 1 0 1 1 1 0 0 0 0 1 0 0
 1 0 0 1 0 1 0 1 1 1 0 1 0 0 1 0 1 1 0 1 1 1 1 0 1 1 0 0 1 0 1 1 0 1 1 0 1
 0 1 0 0 1 0 1 1 1 0 0 0 1 0 0 1 1 0 1 0 0 1 0 1 1 0 0 0 1 1 0 0 1 1 0 0 1
 0 0 0 1 0 0 0 1 0 1 1 1 0 0 0 0 0 0 0 1 1 0 0 0 1 1 0 1 0 1 0 0 1 1 0 0 1
 1 1 0 1 1 0 0 1 0 0 1 0 0 1 0 0 0 0 1 0 1 1 1 1 1 1 0 1 0 0 0 0 1 1 0 1 1
 1 0 1 0 0 0 1 1 1 0 1 1 1 1 0 1 1 0 1 0 1 1 0 0 0 0 1 0 1 0 1 1 0 0 0 1 0
 1 1 0 0 0 0 0 0 1 0 1 0 0 0 1 0 1 1 0 1 1 0 1 1 1 1 1 1 0 0 0 1 0 0 0 0 1
 0 0 0 1 0 1 0 1 0 0 0 0 1 0 1 1 0 1 1 0 1 1 0 0 0 0 1 0 0 0 1 1 1 1 1 1 1
 1 1 0 1 0 1 0 0 1 0 1 1 1 1 0 1 1 0 1 1 1 1 0 1 1 1 1 1 0 0 0 1 1 0 1 1 1
 1 0 1 1 0 0 0 1 1 1 0 0 1 0 0 0 1 0 1 0 1 1 1 1 0 0 0 1 0 1 0 0 0 0 0 1 1
 1 0 1 0 1 1 0 1 0 0 0 0 

In [5]:
print(test_y)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 

## data reshape & split

In [6]:
train_X=np.reshape(train_X, ( 580,750,1))
test_X=np.reshape(test_X, ( 562,750,1))
print(train_X.shape, train_y.shape)

(580, 750, 1) (580,)


In [7]:
train_y = to_categorical(train_y,2)
test_y = to_categorical(test_y,2)
print(type(train_y),train_y.shape)
print(test_y.shape)

<class 'numpy.ndarray'> (580, 2)
(562, 2)


In [8]:
print(test_y[250:562])
#print(test_y[0:250])

[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 

In [8]:
train_X,val_X,train_y,val_y=train_test_split(train_X,train_y,test_size=0.4,shuffle=True,stratify=train_y,random_state=33)

# CNN

In [15]:
def cnn():
    model=Sequential()
    model.add(Conv1D(activation='relu',input_shape=(750,1),strides=2,filters=8,kernel_size=3))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size = 2))
    
    model.add(Conv1D(activation='relu',strides=2,filters=16,kernel_size=3))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size = 2))
    
    model.add(Conv1D(activation='relu',strides=2,filters=32,kernel_size=3))
    model.add(BatchNormalization())
    model.add(Conv1D(activation='relu',strides=1,filters=32,kernel_size=1))
    model.add(BatchNormalization())
    
    model.add(Conv1D(activation='relu',strides=2,filters=64,kernel_size=3))
    model.add(BatchNormalization())
    model.add(Conv1D(activation='relu',strides=1,filters=64,kernel_size=1))
    model.add(BatchNormalization())
    #model.add(MaxPooling1D(pool_size = 2))
    
    model.add(Flatten())
    model.add(Dense(32, activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(16, activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2,activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=["accuracy"])
    return model
model=cnn()
model.summary()
my_callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=10),
    tf.keras.callbacks.ModelCheckpoint(filepath='model.{epoch:02d}-{val_loss:.2f}.h5',monitor='val_loss',   # val_loss 값이 개선되었을때 호출됩니다
                             verbose=1,            # 로그를 출력합니다
                             save_best_only=True,  # 가장 best 값만 저장합니다
                             mode='auto'           # auto는 알아서 best를 찾습니다. min/max
                            )
]
model.fit(train_X,train_y,epochs=50,callbacks=my_callbacks,verbose=1,batch_size=1,validation_data=(val_X,val_y))

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_6 (Conv1D)            (None, 374, 8)            32        
_________________________________________________________________
batch_normalization_6 (Batch (None, 374, 8)            32        
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 187, 8)            0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 93, 16)            400       
_________________________________________________________________
batch_normalization_7 (Batch (None, 93, 16)            64        
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 46, 16)            0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 22, 32)           

In [16]:
from tensorflow.keras.models import load_model
model = load_model('model.04-0.20_SGD.h5')

performance_test = model.evaluate(test_X, test_y, batch_size=1)
print('Test Loss and Accuracy -> ', performance_test)

562/1 [=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

Test Loss and Accuracy ->  [1.0542990295572698, 0.71352315]


## 성능평가 결과
- validation 0.4 로 했을때 model.06-0.16.h5
    - loss & acc => 0.49 & 0.89
- SGD로 했을때 model.04-0.20_SGD.h5
    - 1.05 & 0.71

In [14]:
result=model.predict(test_X,batch_size=1)
print(result[530:])
print("============================")
print(test_y[530:])

[[3.5166988e-06 9.9999988e-01]
 [1.0148901e-01 9.5175254e-01]
 [1.2446346e-06 1.0000000e+00]
 [1.3343046e-02 9.9756515e-01]
 [5.7274121e-04 9.9993801e-01]
 [1.0907582e-03 9.9983251e-01]
 [1.9844039e-03 9.9975520e-01]
 [9.9984384e-01 1.6689177e-05]
 [9.9480671e-01 1.8463597e-03]
 [2.8287157e-04 9.9997425e-01]
 [9.9119824e-01 3.8491113e-03]
 [4.5966062e-05 9.9999702e-01]
 [2.9248154e-02 9.9138016e-01]
 [2.3068315e-01 8.6303610e-01]
 [1.4719785e-03 9.9979132e-01]
 [5.5057731e-06 9.9999988e-01]
 [0.0000000e+00 1.0000000e+00]
 [2.8240223e-07 1.0000000e+00]
 [6.1303942e-04 9.9992800e-01]
 [9.0250809e-04 9.9990261e-01]
 [7.7552795e-06 9.9999976e-01]
 [2.4267688e-06 9.9999988e-01]
 [7.8118426e-08 1.0000000e+00]
 [8.1564259e-08 1.0000000e+00]
 [4.7615645e-06 9.9999988e-01]
 [2.6856247e-05 9.9999869e-01]
 [9.0645878e-03 9.9824202e-01]
 [9.6986247e-03 9.9764758e-01]
 [1.0074449e-01 9.5912808e-01]
 [9.7225589e-01 1.2073341e-02]
 [7.8419000e-01 2.0631687e-01]
 [9.5776886e-01 1.3633308e-02]]
[[0. 1.

# DNN

In [29]:
from tensorflow.keras import layers, models

class DNN(models.Sequential):
    def __init__(self, Nout):
        super().__init__()

        #self.add(layers.Dense(100, activation='relu', input_shape=(750,), name='Hidden-1'))
        self.add(layers.Dense(50, activation='relu', name='Hidden-2'))
        self.add(layers.Dense(Nout, activation='sigmoid'))
        self.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#def main():
number_of_class = 2
Nout = number_of_class

model = (DNN(Nout))
    
#history = model.fit(train_X, train_y, epochs=50, batch_size=16, validation_split=0.01)

my_callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=2),
    tf.keras.callbacks.ModelCheckpoint(filepath='model.{epoch:02d}-{val_loss:.2f}.h5')
]
model.fit(train_X, train_y, epochs=10, callbacks=my_callbacks, batch_size = 1, validation_split=0.95)

Train on 29 samples, validate on 551 samples
Epoch 1/10
29/29 [==============================] - 1s 40ms/sample - loss: 0.6785 - accuracy: 0.9660 - val_loss: 0.6962 - val_accuracy: 0.4610
Epoch 2/10
29/29 [==============================] - 1s 25ms/sample - loss: 0.6431 - accuracy: 1.0000 - val_loss: 0.7017 - val_accuracy: 0.4610
Epoch 3/10
29/29 [==============================] - 1s 24ms/sample - loss: 0.5984 - accuracy: 1.0000 - val_loss: 0.7116 - val_accuracy: 0.4610


In [32]:
performance_test = model.evaluate(test_X, test_y, batch_size=1)
print('Test Loss and Accuracy -> ', performance_test)

562/1 [=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

Test Loss and Accuracy ->  [0.15935717464766044, 1.0]


# VGG